In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from scipy.ndimage import median_filter
from scipy.ndimage import uniform_filter1d
import lightkurve as lk

In [ ]:
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["figure.dpi"] = 400

In [ ]:
f = "exoplanets/tess-s0010-1-1_173.957400_-29.156000_100x100_astrocut.fits"
f2 = "exoplanets/tess-s0001-1-4_316.631902_-26.692632_64x64_astrocut.fits"

In [ ]:
exo = tess_cpm.Source(f)

In [ ]:
# exo.plot_cutout();

In [ ]:
exo.set_aperture(rowlims=[49, 50], collims=[49, 52])
exo.plot_cutout(rowlims=[45, 55], collims=[45, 55], show_aperture=True);

In [ ]:
# exo.plot_pix_by_pix();

In [ ]:
exo.remove_poly_model()
exo.add_cpm_model()
exo.set_regs([0.1])
exo.holdout_fit_predict(k=100);
no_poly_dt_lc = exo.get_aperture_lc("cpm_subtracted_flux")

exo.add_cpm_model()
exo.add_poly_model()
exo.set_regs([0.1, 0.01])
exo.holdout_fit_predict(k=100);
with_poly_dt_lc = exo.get_aperture_lc("cpm_subtracted_flux")
with_poly_sub_poly_dt_lc = with_poly_dt_lc - exo.get_aperture_lc("poly_model_prediction")

In [ ]:
plt.plot(exo.time, no_poly_dt_lc, label="No Polynomial")
plt.plot(exo.time, with_poly_dt_lc, label="With Polynomial")
plt.plot(exo.time, with_poly_sub_poly_dt_lc, label="With Polynomial (subtract poly)")

plt.legend()

In [ ]:
# exo.plot_pix_by_pix(split=True, data_type="raw")
# exo.plot_pix_by_pix(split=True, data_type="prediction")
# exo.plot_pix_by_pix(split=True, data_type="cpm_prediction")
# exo.plot_pix_by_pix(split=True, data_type="poly_model_prediction")
exo.plot_pix_by_pix(split=True, data_type="cpm_subtracted_flux")
# no_poly_dt_lc = exo.get_aperture_lc("cpm_subtracted_flux")
# with_poly_dt_lc = exo.get_aperture_lc("cpm_subtracted_flux") - exo.get_aperture_lc("poly_model_prediction")
with_poly_dt_lc = exo.get_aperture_lc("cpm_subtracted_flux")

In [ ]:
# lc.shape
outliers = exo.get_outliers(window=10, sigma_upper=5, sigma_lower=0.8)

In [ ]:
plt.plot(exo.time, lc)
plt.plot(exo.time[outliers], lc[outliers], "x", c="red")
plt.plot(exo.time[~outliers], lc[~outliers], "-", c="C2")

In [ ]:
exo.holdout_fit_predict(k=100, mask=~outliers);
exo.plot_pix_by_pix("cpm_prediction");
exo.plot_pix_by_pix("cpm_subtracted_flux");

In [ ]:
masked_lc = exo.get_aperture_lc("cpm_subtracted_flux")
# plt.plot(exo.time, lc)
plt.plot(exo.time, masked_lc, alpha=0.7)
plt.plot(exo.time, lc, alpha=0.5)

In [ ]:
apt_cpm = exo.get_aperture_lc(data_type="cpm_prediction")
apt_lc = exo.get_aperture_lc(data_type="rescaled_cpm_subtracted_flux")
# exo.get_aperture_lc(split=False, data_type="cpm_subtracted_lc")

In [ ]:
# plt.plot(exo.time, apt_cpm, ".")
plt.plot(exo.time, apt_lc, ".")
# outliers = np.abs(apt_lc) > 1.5*np.std(apt_lc)
# print(np.sum(outliers))
# plt.plot(exo.time[outliers], apt_lc[outliers], "x", c="k")
# plt.plot(exo.time[~outliers], apt_lc[~outliers], "x", c="k")

# apt_lc[pre_clip] = np.nan
# clip = np.abs(apt_lc) > 1*np.std(apt_lc[pre_clip])
# plt.plot(exo.time[clip], apt_lc[clip], "x", c="r")
# np.any(clip)

In [ ]:
cpm_regs = 10.0 ** np.arange(-10, 10)

In [ ]:
min_cpm_reg, cdpps = exo.calc_min_cpm_reg(cpm_regs, k=10)
# plt.savefig("Exoplanet_CDPPS.png", dpi=200)

In [ ]:
cpm_regs

In [ ]:
print(min_cpm_reg)
exo.set_regs([min_cpm_reg])
exo.holdout_fit_predict(k=10);
flux_splits = exo.get_aperture_lc(data_type="raw", split=True)
splits = exo.get_aperture_lc(data_type="cpm_subtracted_flux", split=True)
f, axs = plt.subplots(2, 1, sharex=True, figsize=(18, 8))
for time, split, f_split in zip(exo.split_times, splits, flux_splits):
    axs[0].plot(time, f_split, ".")
    axs[0].set_ylabel("Original Flux", fontsize=15)
    axs[1].plot(time, split)
    axs[1].set_ylabel("Detrended Flux", fontsize=15)
    axs[1].set_xlabel( "Time [BTJD]", fontsize=15)
# plt.savefig("Exoplanet_LC.png", dpi=200)

In [ ]:
print(min_cpm_reg)
exo.set_regs([min_cpm_reg])
exo.holdout_fit_predict(k=10);
flux_splits = exo.get_aperture_lc(data_type="raw", split=True)
splits = exo.get_aperture_lc(data_type="cpm_subtracted_flux", split=True)
f, axs = plt.subplots(2, 1, sharex=True, figsize=(18, 8))
for time, split, f_split in zip(exo.split_times, splits, flux_splits):
    axs[0].plot(time, f_split, ".")
    axs[0].set_ylabel("Original Flux", fontsize=15)
    axs[1].plot(time, split)
    axs[1].set_ylabel("Detrended Flux", fontsize=15)
    axs[1].set_xlabel( "Time [BTJD]", fontsize=15)
# plt.savefig("Exoplanet_LC.png", dpi=200)

In [ ]:
lc = lk.TessLightCurve(time=exo.time[~outliers], flux=apt_lc[~outliers])
# lc.plot(normalize=False)
# lc.normalize()
# np.nanmedian(lc.flux)
# np.nanstd(lc.flux)
# np.abs(np.nanmedian(lc.flux)) < 0.5*np.nanstd(lc.flux)
lc.plot()
lc.estimate_cdpp()
# a = lc.flatten()
# a

In [ ]:
# split_cpm_subtracted_lc = exo.get_aperture_lc(split=True, data_type="cpm_subtracted_lc")
# cpm_subtracted_lc = exo.get_aperture_lc(data_type="cpm_subtracted_lc")

# for t, l in zip(exo.split_times, split_cpm_subtracted_lc):
#     plt.plot(t, l)

## TIC 29857954

In [ ]:
# TIC 29857954
tic = tess_cpm.Source(f2, remove_bad=True)
# tic.set_aperture(rowlims=[28, 36], collims=[28, 36])  # full aperture
tic.set_aperture(rowlims=[32, 34], collims=[31, 33])  # not full aperture
# tic.set_aperture(rowlims=[32, 33], collims=[32, 32])  # central pixel

tic.plot_cutout(show_aperture=True, h=99);
tic.add_cpm_model(n=256)
# tic.add_poly_model()
tic.set_regs([0.01])
tic.holdout_fit_predict(k=150);
tic.plot_pix_by_pix();
# tic.plot_pix_by_pix("normalized_flux");
# tic.plot_pix_by_pix("cpm_prediction", split=True);

# tic.plot_pix_by_pix("cpm_subtracted_flux", split=True);
# tic.plot_pix_by_pix("poly_model_prediction", split=True);

# no_mask_tic_lc = tic.get_aperture_lc("cpm_subtracted_flux")
no_mask_tic_lc = tic.get_aperture_lc("rescaled_cpm_subtracted_flux")

plt.plot(tic.time, no_mask_tic_lc)

# There's already one known asteroid crossing in the aperture for this source around 1338.
# Is there another one around the top left corner at 1347?

In [ ]:
# TIC 29857954
# With Polynomial
with_poly = tess_cpm.Source(f2, remove_bad=True)
# with_poly.set_aperture(rowlims=[28, 36], collims=[28, 36])  # full aperture
with_poly.set_aperture(rowlims=[32, 34], collims=[31, 33])  # not full aperture
# with_poly.set_aperture(rowlims=[33, 33], collims=[32, 32])  # central pixel

# with_poly.plot_cutout(show_aperture=True);
with_poly.add_cpm_model(n=256)
with_poly.add_poly_model()
with_poly.set_regs([0.01, 0.01])
with_poly.holdout_fit_predict(k=150);
# with_poly.plot_pix_by_pix();
# with_poly.plot_pix_by_pix("normalized_flux");
# with_poly.plot_pix_by_pix("cpm_prediction", split=True);

# with_poly.plot_pix_by_pix("cpm_subtracted_flux", split=True);
# with_poly.plot_pix_by_pix("poly_model_prediction", split=True);


detrended_lc_with_poly = with_poly.get_aperture_lc("cpm_subtracted_flux")
poly_pred = with_poly.get_aperture_lc("poly_model_prediction")
intercept_pred = with_poly.get_aperture_lc("intercept_prediction")
plt.plot(with_poly.time, detrended_lc_with_poly)
plt.plot(with_poly.time, detrended_lc_with_poly-poly_pred)

# There's already one known asteroid crossing in the aperture for this source around 1338.
# Is there another one around the top left corner at 1347?

In [ ]:
plt.plot(with_poly.time, detrended_lc_with_poly, alpha=0.9, lw=1, label="CPM Subtracted Flux (Simultaneous Fit with Poly Model)")
plt.plot(with_poly.time, detrended_lc_with_poly-poly_pred, alpha=0.9, lw=1, label="CPM&Poly Subtracted Flux (Simultanesous Fit with Poly Model)")
# plt.plot(tic.time, no_mask_tic_lc, alpha=0.6, lw=1, label="CPM Subtracted Flux (No Poly Model)")
plt.legend(fontsize=10, markerscale=2)
# plt.savefig("TIC29857954_different_model_lcs.pdf", bbox_inches="tight")

## Plot Outliers & De-trend without them

In [ ]:
no_mask_tic_lc.shape

outliers = tic.get_outliers(sigma_upper=1.5, sigma_lower=1.5, window=30)
plt.plot(tic.time, no_mask_tic_lc)
# s = (tic.time > 1347) * (tic.time < 1350) + (tic.time > 1353)
# 338<Time<1339 and 1346.8<Time<1348.6
# s = ((tic.time > 1338) * (tic.time < 1339)) + ((tic.time > 1346.8) * (tic.time < 1348.6))
# asteroid = ((tic.time > 1338)) * (tic.time < 1339)
# outliers[asteroid] = True

# not_outliers = tic.time > 1347.
# outliers[not_outliers] = False

plt.plot(tic.time[outliers], no_mask_tic_lc[outliers], "x", c="red")
plt.plot(tic.time[~outliers], no_mask_tic_lc[~outliers], c="green")

In [ ]:
tic.holdout_fit_predict(k=200, mask=~outliers);
# tic.plot_pix_by_pix("rescaled_cpm_subtracted_flux", split=True)

In [ ]:
# tic_lc = tic.get_aperture_lc("rescaled_cpm_subtracted_flux")
mask_tic_lc = tic.get_aperture_lc("rescaled_cpm_subtracted_flux")
# plt.plot(tic.time[~s], tic_lc[~s])
plt.plot(tic.time, mask_tic_lc / np.median(mask_tic_lc), alpha=0.3)
plt.plot(tic.time, no_mask_tic_lc / np.nanmedian(no_mask_tic_lc), alpha=0.3)

# plt.ylim([0.995, 1.005])

In [ ]:
plt.figure(figsize=(16, 10))

plt.plot(tic.time, mask_tic_lc, c="k")
plt.xlabel("Time - 2457000 [Days]", fontsize=30)
plt.ylabel("Renormalized CPM Flux", fontsize=30)
plt.title("TOI-172 (TIC 29857954)", y=1, fontsize=40)
plt.tick_params(labelsize=20)
# plt.ylim([0.995, 1.005])
# plt.savefig("TOI172.png", dpi=300)

In [ ]:
# lc.to_csv("/home/soichiro/code/tessninja3/cpm_lc.csv")

In [ ]:
# transits = tic.get_outliers(sigma_lower=1.5, sigma_upper=10, window=20)
# # tic.get_outliers?
# plt.plot(tic.time, tic_lc, "k")
# plt.plot(tic.time[transits], tic_lc[transits], "x", c="r")

In [ ]:
coarse1 = (tic.time > 1338) & (tic.time < 1338.5)
# coarse2 = (tic.time > 1346.8) & (tic.time < 1348.6) 
coarse2 = (tic.time > 1347) & (tic.time < 1349.5) 

coarse = coarse1 + coarse2

half_duration = 0.2
period = 9.48
t1 = 1326.925
transit1 = (tic.time > (t1-half_duration)) & (tic.time < (t1+half_duration))
transit2 = (tic.time > (t1+period-half_duration)) & (tic.time < (t1+period+half_duration))
transit3 = (tic.time > (t1+2*period-half_duration)) & (tic.time < (t1+2*period+half_duration))
transits = transit1 + transit2 + transit3

end = tic.time > 1353

flag = coarse + transits + (tic.time > 1353)
dt_mask = coarse1 + transits

In [ ]:
plt.plot(tic.time, mask_tic_lc, "k.")
# plt.plot(tic.time[coarse], tic_lc[coarse], "x", c="r")
# plt.plot(tic.time[transits], tic_lc[transits], "x", c="r")
plt.plot(tic.time[flag], mask_tic_lc[flag], "x", c="r")

In [ ]:
tic.holdout_fit_predict(k=150, mask=~dt_mask);
# tic_lc = tic.get_aperture_lc("rescaled_cpm_subtracted_flux")
mask_tic_lc = tic.get_aperture_lc("rescaled_cpm_subtracted_flux")
# plt.plot(tic.time[~s], tic_lc[~s])
plt.plot(tic.time, mask_tic_lc / np.median(mask_tic_lc), alpha=0.3, marker=".")
plt.plot(tic.time, no_mask_tic_lc / np.median(no_mask_tic_lc), alpha=0.3, marker=".")

plt.ylim([0.995, 1.005])

In [ ]:
plt.plot(tic.time[transit1], mask_tic_lc[transit1] / np.median(mask_tic_lc), alpha=0.3, marker=".", label="Masked")
plt.plot(tic.time[transit1], no_mask_tic_lc[transit1] / np.median(no_mask_tic_lc), alpha=0.3, marker=".", label="No Mask")
plt.legend()
plt.ylim([0.995, 1.005])

In [ ]:
plt.plot(tic.time[transit2], mask_tic_lc[transit2] / np.median(mask_tic_lc), alpha=0.3, marker=".", label="Masked")
plt.plot(tic.time[transit2], no_mask_tic_lc[transit2] / np.median(no_mask_tic_lc), alpha=0.3, marker=".", label="No Mask")
plt.legend()
plt.ylim([0.995, 1.005])

In [ ]:
plt.plot(tic.time[transit3], mask_tic_lc[transit3] / np.median(mask_tic_lc), alpha=0.3, marker=".", label="Masked")
plt.plot(tic.time[transit3], no_mask_tic_lc[transit3] / np.median(no_mask_tic_lc), alpha=0.3, marker=".", label="No Mask")
plt.legend()
plt.ylim([0.995, 1.005])

In [ ]:
lc = lk.TessLightCurve(time=tic.time, flux=mask_tic_lc / np.nanmedian(mask_tic_lc))
lc.flatten(window_length=51).plot()
lc.estimate_cdpp(transit_duration=2, sigma=5.0)

In [ ]:
# lc = lk.TessLightCurve(time=tic.time[~flag], flux=mask_tic_lc[~flag] / np.nanmedian(mask_tic_lc[~flag]))
lc = lk.TessLightCurve(time=tic.time[~flag], flux=mask_tic_lc[~flag])
lc.plot()
lc.estimate_cdpp(transit_duration=2, savgol_window=51)
# lc.flatten().estimate_cdpp(transit_duration=2)

In [ ]:
# plt.plot(tic.time, tic_lc+1)

In [ ]:
# lc = lk.TessLightCurve(time=tic.time[~s], flux=(tic_lc)[~s] / np.nanmedian(tic_lc[~s]))
# lc = lk.TessLightCurve(time=tic.time[~asteroid], flux=(tic_lc[~asteroid] / np.nanmedian(tic_lc[~asteroid])))

# lc = lk.TessLightCurve(time=tic.time[~outliers], flux=(tic_lc+1)[~outliers])

In [ ]:
lc.plot()
cleaned_lc = lc.remove_outliers(sigma=5)
cleaned_lc.plot()

In [ ]:
eleanor_time = np.loadtxt("/home/soichiro/code/tessninja3/toi172_eleanor_lc.csv", delimiter=",", usecols=1, skiprows=1)
eleanor_flux = np.loadtxt("/home/soichiro/code/tessninja3/toi172_eleanor_lc.csv", delimiter=",", usecols=2, skiprows=1)

In [ ]:
print(eleanor_time.size, eleanor_flux.size)

In [ ]:
eleanor_coarse1 = (eleanor_time > 1338) & (eleanor_time < 1338.5)
eleanor_coarse2 = (eleanor_time > 1347) & (eleanor_time < 1349.5) 
eleanor_coarse = eleanor_coarse1 + eleanor_coarse2

half_duration = 0.2
period = 9.48
t1 = 1326.925
eleanor_transit1 = (eleanor_time > (t1-half_duration)) & (eleanor_time < (t1+half_duration))
eleanor_transit2 = (eleanor_time > (t1+period-half_duration)) & (eleanor_time < (t1+period+half_duration))
eleanor_transit3 = (eleanor_time > (t1+2*period-half_duration)) & (eleanor_time < (t1+2*period+half_duration))
eleanor_transits = eleanor_transit1 + eleanor_transit2 + eleanor_transit3

eleanor_flag = eleanor_coarse + eleanor_transits + (eleanor_time > 1353)
# eleanor_flag = coarse1 + transits

In [ ]:
eleanor_lc_cc = lk.TessLightCurve(time=eleanor_time[~eleanor_flag], flux=eleanor_flux[~eleanor_flag])
eleanor_lc_cc.flatten(window_length=51).plot()
eleanor_lc_cc.estimate_cdpp(transit_duration=2, savgol_window=51)

In [ ]:
print(eleanor_time.size, tic.time.size)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

ax.plot(eleanor_time, eleanor_flux / np.nanmedian(eleanor_flux), ".", color="C1", label="eleanor 1-hr CDPP: 369 ppm")
ax.plot(tic.time, mask_tic_lc / np.nanmedian(mask_tic_lc), ".", color="C0", label="unpopular 1-hr CDPP: 273 ppm")
ax.legend(edgecolor="k", loc=2, fontsize=20, markerscale=3)
# ax.set_ylim((0.996, 1.001))

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

ax.plot(eleanor_time, eleanor_flux / np.nanmedian(eleanor_flux)+0.01, color="C1", label="eleanor 1-hr CDPP: 367 ppm")
ax.plot(tic.time, mask_tic_lc / np.nanmedian(mask_tic_lc), color="C0", label="unpopular 1-hr CDPP: 274 ppm")
# ax.plot(eleanor_time[eleanor_time < 1353], eleanor_flux[eleanor_time < 1353] / np.nanmedian(eleanor_flux[eleanor_time < 1353])+0.01, 
#         color="C1", label="eleanor 1-hr CDPP: 369 ppm")
# ax.plot(tic.time[tic.time < 1353], mask_tic_lc[tic.time < 1353] / np.nanmedian(mask_tic_lc[tic.time < 1353]), 
#         color="C0", label="unpopular 1-hr CDPP: 273 ppm")
ax.legend(edgecolor="k", loc=2, fontsize=20, markerscale=3)

hd = 0.2
op = 9.48
t1 = 1326.925
t2 = t1+op
t3 = t2+op
color="C4"
alpha=0.3
ax.axvspan(t1-hd, t1+hd, facecolor=color, alpha=alpha)
ax.axvspan(t2-hd, t2+hd, facecolor=color, alpha=alpha)
ax.axvspan(t3-hd, t3+hd, facecolor=color, alpha=alpha)

ax.axvspan(1338, 1338.5, facecolor="Gray", alpha=0.3)
ax.axvspan(1347, 1349.5, facecolor="Gray", alpha=0.3)
ax.axvspan(1353, tic.time[-1]+0.1, facecolor="Gray", alpha=0.3)
ax.set_xlabel("Time [BJD-2457000]", fontsize=20)
ax.set_ylabel("Normalized Flux + Constant", fontsize=20)
ax.tick_params(labelsize=20)
# plt.legend(fontsize=20, markerscale=3, edgecolor="k")
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
# plt.ylim([0.995, 1.005])
plt.savefig("toi172_comparison.pdf", bbox_inches="tight")

In [ ]:
plt.plot(eleanor_time[eleanor_transit1], eleanor_flux[eleanor_transit1] / np.nanmedian(eleanor_flux), ".-", color="C1", label="eleanor")
plt.plot(tic.time[transit1], mask_tic_lc[transit1] / np.nanmedian(mask_tic_lc), ".-", color="C0", label="unpopular")
plt.legend()

In [ ]:
plt.plot(eleanor_time[eleanor_transit2], eleanor_flux[eleanor_transit2] / np.nanmedian(eleanor_flux), ".-", color="C1", label="eleanor")
plt.plot(tic.time[transit2], mask_tic_lc[transit2] / np.nanmedian(mask_tic_lc), ".-", color="C0", label="unpopular")
plt.legend()
plt.gca().get_xaxis().get_major_formatter().set_useOffset(False)

In [ ]:
plt.plot(eleanor_time[eleanor_transit3], eleanor_flux[eleanor_transit3] / np.nanmedian(eleanor_flux), ".-", color="C1", label="eleanor")
plt.plot(tic.time[transit3], mask_tic_lc[transit3] / np.nanmedian(mask_tic_lc), ".-", color="C0", label="unpopular")
plt.legend()

In [ ]:
fig, axs = plt.subplots(figsize=(16, 3), nrows=1, ncols=3, sharey=True)
# fig.subplots_adjust(hspace=0.0, wspace=0)
axs[0].plot(eleanor_time[eleanor_transit1], eleanor_flux[eleanor_transit1] / np.nanmedian(eleanor_flux), ".-", color="C1", label="eleanor")
axs[0].plot(tic.time[transit1], mask_tic_lc[transit1] / np.nanmedian(mask_tic_lc), ".-", color="C0", label="unpopular")
# axs[0].legend()
axs[1].plot(eleanor_time[eleanor_transit2], eleanor_flux[eleanor_transit2] / np.nanmedian(eleanor_flux), ".-", color="C1", label="eleanor")
axs[1].plot(tic.time[transit2], mask_tic_lc[transit2] / np.nanmedian(mask_tic_lc), ".-", color="C0", label="unpopular")
axs[1].get_xaxis().get_major_formatter().set_useOffset(False)
axs[2].plot(eleanor_time[eleanor_transit3], eleanor_flux[eleanor_transit3] / np.nanmedian(eleanor_flux), ".-", color="C1", label="eleanor")
axs[2].plot(tic.time[transit3], mask_tic_lc[transit3] / np.nanmedian(mask_tic_lc), ".-", color="C0", label="unpopular")

axs[0].set_ylabel("Normalized Flux", fontsize=15)
axs[1].set_xlabel("Time [BJD-2457000]", fontsize=15)
for ax in axs:
    ax.tick_params(labelsize=20)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1.5)

In [ ]:
eleanor_fold = lk.TessLightCurve(time=eleanor_time, flux=eleanor_flux / np.nanmedian(eleanor_flux)).fold(9.48, t0=1326.92)
unpopular_fold = lk.TessLightCurve(time=tic.time, flux=mask_tic_lc / np.nanmedian(mask_tic_lc)).fold(9.48, t0=1326.92)

In [ ]:
eleanor_phase = (eleanor_fold.phase.value > -0.3) * (eleanor_fold.phase.value < 0.3)
unpopular_phase = (unpopular_fold.phase.value > -0.3) * (unpopular_fold.phase.value < 0.3)

plt.plot(eleanor_fold.phase.value[eleanor_phase], eleanor_fold.flux.value[eleanor_phase], ".C1")
# plt.plot(eleanor_fold.phase.value[eleanor_phase], uniform_filter1d(eleanor_fold.flux.value[eleanor_phase], size=10), "*C0")
plt.plot(eleanor_fold.phase.value[eleanor_phase], median_filter(eleanor_fold.flux.value[eleanor_phase], size=12), "-C1")
plt.plot(unpopular_fold.phase.value[unpopular_phase], unpopular_fold.flux.value[unpopular_phase], ".C0")
# plt.plot(unpopular_fold.phase.value[unpopular_phase], uniform_filter1d(unpopular_fold.flux.value[unpopular_phase], size=10), "*C1")
plt.plot(unpopular_fold.phase.value[unpopular_phase], median_filter(unpopular_fold.flux.value[unpopular_phase], size=12), "-C0")

In [ ]:
eleanor_flux

In [ ]:
end = tic.time > 1353
plt.plot(tic.time[~end], mask_tic_lc[~end] / np.nanmedian(mask_tic_lc[~end]))

In [ ]:
window = (eleanor_time < 1350) * (eleanor_time > 1346)
plt.plot(eleanor_time[window], eleanor_flux[window], ".")